In [21]:
import json
import pandas as pd
import re

with open("./matches/EFBO2372905.json") as f:
    data = f.read()

data = json.loads(data)
match_key = next(iter(data["matchData"][0]["tournamentDatesWithEvents"]))

comp_data = data["matchData"][0]["tournamentMeta"]
cup_data = data["matchData"][0]["tournamentDatesWithEvents"][match_key][0]["round"]
match_data = data["matchData"][0]["tournamentDatesWithEvents"][match_key][0]["events"][0]

In [22]:
match_data.keys()

dict_keys(['eventKey', 'startTime', 'isTBC', 'minutesElapsed', 'minutesIntoAddedTime', 'eventStatus', 'eventStatusNote', 'eventStatusReason', 'eventOutcomeType', 'eventType', 'seriesWinner', 'cpsId', 'cpsLive', 'homeTeam', 'awayTeam', 'eventProgress', 'venue', 'officials', 'tournamentInfo', 'eventActions', 'startTimeInUKHHMM', 'comment', 'href', 'tournamentName', 'tournamentSlug'])

In [23]:
if match_data["homeTeam"]["name"]["full"] == "Tranmere Rovers":
    tranmere = "homeTeam"
    opponent = "awayTeam"
else:
    tranmere = "awayTeam"
    opponent = "homeTeam"

In [24]:
game_date = match_data["startTime"][:10]
game_date

'2023-08-08'

In [25]:
opposition = match_data[opponent]["name"]["full"]
opposition

'Barnsley'

In [26]:
if match_data["venue"]["name"]["full"] == "Wembley Stadium":
    venue = "N"
elif match_data["venue"]["name"]["full"] == "Prenton Park":
    venue = "H"
elif match_data["venue"]["name"]["full"] != "Prenton Park":
    venue = "H"
venue

'H'

In [27]:
goals_for = match_data[tranmere]["scores"]["score"]
goals_for

2

In [28]:
goals_against = match_data[opponent]["scores"]["score"]
goals_against

2

In [29]:
score = f"{goals_for}-{goals_against}"
score

'2-2'

In [30]:
if goals_for > goals_against:
    outcome = "W"
elif goals_for < goals_against:
    outcome = "L"
else:
    outcome = "D"
outcome

'D'

In [31]:
goal_diff = goals_for - goals_against
goal_diff

0

In [32]:
if comp_data["tournamentName"]["first"] in ["League One", "League Two", "National League"]:
    if "Play-offs" in cup_data["round"]["full"]:
        game_type = "League"
    else:
        game_type = "League Play-Off"
else:
    game_type = "Cup"
game_type

'Cup'

In [33]:
# competition
competition = comp_data["tournamentName"]["first"]
competition

'EFL Cup'

In [34]:
# generic_comp

In [35]:
# league_tier

In [36]:
ko_time = match_data["startTimeInUKHHMM"]
ko_time

'19:45'

In [37]:
# cup_round

In [38]:
cup_leg = None
if " leg" in match_data["eventType"]:
    cup_leg = match_data["eventType"][:1]
cup_leg

In [39]:
cup_stage = cup_data["name"] ["full"]
cup_stage

'First Round North'

In [40]:
cup_replay = match_data["eventType"]

if cup_replay.upper() == "REPLAY":
    cup_replay = 1
else:
    cup_replay = None

cup_replay

In [41]:
cup_section = None
if re.search(r"North(?:ern)?", cup_stage):
    cup_section = re.search(r"North(?:ern)?", cup_stage).group(0)
cup_section

'North'

In [42]:
if match_data["eventProgress"] == "EXTRATIMECOMPLETE":
    aet = 1
else:
    aet = None
aet

In [43]:
pen_gf = int(match_data[tranmere]["scores"]["shootout"])
pen_gf

7

In [44]:
pen_ga = int(match_data[opponent]["scores"]["shootout"])
pen_ga

6

In [45]:
if pen_gf > pen_ga:
    pen_outcome = "W"
elif pen_gf < pen_ga:
    pen_outcome = "L"
else:
    None
pen_outcome

'W'

In [46]:
pen_score = f"{pen_gf}-{pen_ga}"
pen_score

'7-6'

In [47]:
agg_gf = match_data[tranmere]["scores"]["aggregate"]
agg_gf

In [48]:
agg_ga = match_data[opponent]["scores"]["aggregate"]
agg_ga

In [49]:
agg_outcome = None
if agg_gf:
    if agg_gf > agg_ga:
        agg_outcome = "W"
    elif agg_gf < agg_ga:
        agg_outcome = "L"
    else:
        agg_outcome = "D"
agg_outcome        

In [50]:
agg_score = None
if agg_outcome:
    agg_score = f"{agg_gf}-{agg_ga}"
agg_score

In [51]:
decider = match_data["eventOutcomeType"]

if decider == "shootout":
    decider = "pens"
if decider == "extra-time":
    decider = "extra time"

In [52]:
if aet or pen_outcome or agg_outcome:
    cup_outcome = match_data[tranmere]["eventOutcome"].upper()[:1]
else:
    cup_outcome = None
cup_outcome

'W'

In [53]:
if pen_outcome and agg_outcome:
    if pen_outcome == "W":
        str_outcome = "Won"
    else:
        str_outcome = "Lost"
    outcome_desc = f"{agg_score}. {str_outcome} {pen_score} on pens"
elif pen_outcome and not agg_outcome:
    if pen_outcome == "W":
        str_outcome = "Won"
    else:
        str_outcome = "Lost"
    outcome_desc = f"{str_outcome} {pen_score} on pens"
elif agg_outcome and not pen_outcome:
    if agg_outcome == "W":
        str_outcome = "Won"
    else:
        str_outcome = "Lost"
    outcome_desc = f"{str_outcome} {agg_score} on agg"
else:
    outcome_desc = None
    
outcome_desc

'Won 7-6 on pens'

In [54]:
# game_length
if aet == 1:
    game_length = 120
else:
    game_length = 90
game_length

90

In [55]:
stadium = match_data["venue"]["name"]["full"]
stadium

'Oakwell'

In [62]:
teams = ["homeTeam", "awayTeam"]

player_actions = pd.DataFrame()
for team in teams:
    team_name = match_data[team]["name"]["full"]
    try:
        players = match_data[team]["playerActions"]
        for player in players:
            player_name = player["name"]["full"]
            
            actions = player["actions"]
            for action in actions:
                action_df = pd.json_normalize(action)
                action_df["game_date"] = pd.to_datetime(game_date)
                action_df["team"] = team_name
                action_df["player_name"] = player
                player_actions = pd.concat([player_actions, action_df], axis = 0)
    except KeyError:
        pass
player_actions

,type,timeElapsed,addedTime,penalty,ownGoal,displayTime,game_date,team,player_name
0,goal,45,3,False,False,45'+3,2023-08-08,Barnsley,NaN
0,goal,90,8,False,False,90'+8,2023-08-08,Barnsley,NaN
0,goal,23,0,False,False,23',2023-08-08,Tranmere Rovers,NaN
0,goal,47,0,False,False,47',2023-08-08,Tranmere Rovers,NaN


In [63]:
teams = ["homeTeam", "awayTeam"]

player_actions = pd.DataFrame()
for team in teams:
    team_name = match_data[team]["name"]["full"]
    try:
        players = match_data[team]["playerActions"]
        for player in players:
            player_name = player["name"]["full"]
            
            actions = player["actions"]
            for action in actions:
                action_df = pd.json_normalize(action)
                action_df["game_date"] = pd.to_datetime(game_date)
                action_df["team"] = team_name
                action_df["player_name"] = player_name
                player_actions = pd.concat([player_actions, action_df], axis = 0)
    except KeyError:
        pass
player_actions

,type,timeElapsed,addedTime,penalty,ownGoal,displayTime,game_date,team,player_name
0,goal,45,3,False,False,45'+3,2023-08-08,Barnsley,Herbie Kane
0,goal,90,8,False,False,90'+8,2023-08-08,Barnsley,Aiden Marsh
0,goal,23,0,False,False,23',2023-08-08,Tranmere Rovers,Luke Norris
0,goal,47,0,False,False,47',2023-08-08,Tranmere Rovers,Samuel Taylor


In [65]:
goals  = player_actions.query("type == 'goal' & team == 'Tranmere Rovers'") \
    .rename(columns = {
        "timeElapsed": "goal_min",
        "ownGoal": "own_goal"
    }).sort_values("goal_min") \
    .drop_duplicates()

goals.penalty = goals.penalty.astype(int)
goals.own_goal = goals.own_goal.astype(int)
goals.loc[goals.own_goal == 1, "player_name"] = "OG"

goals = goals[["game_date", "player_name", "goal_min", "penalty", "own_goal"]]
if not goals.empty:
    # goals.to_csv("goals_updates.csv", index = False)
    print(f"{len(goals)} goals saved.")
else:
    print("No goals found.")

goals

2 goals saved.


,game_date,player_name,goal_min,penalty,own_goal
0,2023-08-08,Luke Norris,23,0,0
0,2023-08-08,Samuel Taylor,47,0,0


In [66]:
red_cards = player_actions.query("type.str.contains('red-card')") \
    .rename(columns = {
        "timeElapsed": "min_so",
    }).drop_duplicates()

red_cards = red_cards[["game_date", "player_name", "min_so"]]
if not red_cards.empty:
    red_cards.to_csv("red_card_updates.csv", index = False)
else:
    print("No red cards found.")

red_cards

No red cards found.


,game_date,player_name,min_so
